In [1]:
!conda activate xai_llm


In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
import numpy as np


In [2]:
def get_input():
    # ideally this reads inputs from a file, now it just takes an input
    return input("Enter a sentence: ")
    
anchor_word_idx = 0 # the index of the interested word
prompts_per_word = 20 # number of generated responses  

sentence = get_input()
print("Sentence: ", sentence)

Sentence:  Tokyo is the capital city of Japan


In [3]:
from models.ChatModel import ChatModel
model_name = "openchat"
model = ChatModel(model_name)
print(f"Model: {model_name}")

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ./models/openchat-3.5-0106.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-0106
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

Model: openchat


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | MATMUL_INT8 = 0 | 
Model metadata: {'general.name': 'openchat_openchat-3.5-0106', 'general.architecture': 'llama', 'llama.context_length': '8192', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '4096', 'llama.block_count': '32', 'llama.feed_forward_length': '14336', 'llama.attention.head_count': '32', 'tokenizer.ggml.eos_token_id': '32000', 'general.file_type': '15', 'llama.attention.head_count_kv': '8', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.freq_base': '10000.000000', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.add_bos_token': 'true', 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.chat_template': "{{ bos_token }}{% for 

In [4]:
# Get responses

from tools.command_generator import generate_prompts, prefix_prompt
from tools.evaluate_response import get_replacements
from tqdm import tqdm

def run_prompts(model, sentence, anchor_idx, prompts_per_word=20):
    prompts = generate_prompts(sentence, anchor_idx)
    all_replacements = []
    for prompt in prompts:
        replacements = []
        for _ in tqdm(
            range(prompts_per_word),
            desc=f"Input: {prompt}",
        ):
            response = model.get_response(
                prefix_prompt(prompt),
            ).strip()
            if response:
                replacement = get_replacements(prompt, response)
                if replacement:
                    replacements.append(replacement)
        if len(replacements) > 0:
            all_replacements.append(replacements)
    return all_replacements

all_responses = run_prompts(model, sentence, anchor_word_idx, prompts_per_word)

Input: [MASK] [MASK] the capital city of Japan:  75%|███████▌  | 15/20 [00:29<00:06,  1.30s/it]

 Response is not valid. ['[mask]', '[mask]', 'the', 'capital', 'city', 'of', 'japan'] ['tokyo']


Input: [MASK] is [MASK] capital city of Japan:  10%|█         | 2/20 [00:04<00:37,  2.08s/it]

 Response is not valid. ['[mask]', 'is', '[mask]', 'capital', 'city', 'of', 'japan'] ['tokyo', 'is', 'japans', 'capital', 'city']


Input: [MASK] is [MASK] capital city of Japan:  20%|██        | 4/20 [00:07<00:29,  1.83s/it]

...

Input: [MASK] is the [MASK] city of Japan:  80%|████████  | 16/20 [00:33<00:08,  2.17s/it]

 Response is not valid. ['[mask]', 'is', 'the', '[mask]', 'city', 'of', 'japan'] ['tokyo', 'is', 'the', 'vibrant', 'metropolis', 'of', 'japan']


Input: [MASK] is the [MASK] city of Japan: 100%|██████████| 20/20 [00:41<00:00,  2.07s/it]


 Response is not valid. ['[mask]', 'is', 'the', '[mask]', 'city', 'of', 'japan'] ['tokyo', 'is', 'the', 'bustling', 'metropolis', 'of', 'japan']


Input: [MASK] is the capital city of [MASK]: 100%|██████████| 20/20 [00:35<00:00,  1.79s/it]


In [5]:
for response in all_responses:
    print(response)

[['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['', ''], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is'], ['tokyo', 'is']]
[['tokyo', 'the'], ['', ''], ['tokyo', 'the'], ['', ''], ['tokyo', 'tokyos'], ['tokyo', '[mask]'], ['tokyo', 'tokyos'], ['tokyo', 'the'], ['tokyo', 'the'], ['tokyo', 'tokyos'], ['tokyo', 'the'], ['tokyo', 'tokyos'], ['tokyo', 'tokyos'], ['tokyo', 'the'], ['tokyo', 'the'], ['tokyo', 'the'], ['tokyo', 'the'], ['tokyo', 'the'], ['tokyo', 'the'], ['tokyo', '[tokyo]']]
[['tokyo', 'largest'], ['osaka', 'second largest'], ['tokyo', 'capital'], ['', ''], ['tokyo', 'bustling capital'], ['kyoto', 'ancient'], ['tokyo', 'capital'], ['osaka', 'second largest'], ['tokyo', 'capital'], ['kyoto', 'ancient capital'], ['tokyo', 'capital'], ['tokyo', 'capital'], ['tokyo', 'capi

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s184075\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def compute_pmi(sentence, all_responses, anchor_word_idx):
    # Tokenizing
    words = word_tokenize(sentence.lower())
    # Combine all responses into a single list of tokens
    all_tokens = [word for response in all_responses for word in word_tokenize(str(response).lower())]

    freq_dist = FreqDist(all_tokens)

   
    # Compute PMI for each word
    pmi_scores = {}
    for i, word in enumerate(words):
        if i != anchor_word_idx:
            # Compute co-occurrence counts
            co_occurrence_count = freq_dist[word]

            # Avoid division by zero by adding a small epsilon value
            epsilon = 1e-10

            # Compute PMI
            pmi = np.log2(co_occurrence_count / ((freq_dist[words[anchor_word_idx]] + epsilon) * (freq_dist[word] + epsilon)))
            
            # Min-Max normalization
            pmi_normalized = (pmi - min(pmi_scores.values())) / (max(pmi_scores.values()) - min(pmi_scores.values()))
            
            pmi_scores[word] = pmi_normalized

    return pmi_scores


def highlight_text(sentence, pmi_scores):
    # Tokenize the sentence
    words = word_tokenize(sentence)

    # Highlight words based on PMI scores
    highlighted_sentence = ""
    for word in words:
        if word in pmi_scores:
            # Set highlight color based on PMI score
            pmi_score = pmi_scores[word]
            if pmi_score > 0.7:
                highlighted_word = f"\033[33m{word}\033[0m"  # Bold pink for positive PMI
            elif pmi_score > 0.4:
                highlighted_word = f"\033[34m{word}\033[0m" 
            else:
                highlighted_word = f"\033[35m{word}\033[0m"  # Pink for negative PMI
        else:
            highlighted_word = word
        highlighted_sentence += highlighted_word + " "

    return highlighted_sentence

def highlight_text2(sentence, scores):
    words = word_tokenize(sentence)
    highlighted_sentence = ""
    for i, word in enumerate(words):
        if word in scores:
            score = scores[word]
            if score > 0.5:
                highlighted_word = colored(word, 'orange')
            elif score > 0.3:
                highlighted_word = colored(word, 'blue')
            else:
                highlighted_word = colored(word, 'green')  # Using green as the third color
        else:
            highlighted_word = word
        highlighted_sentence += highlighted_word + " "
    return highlighted_sentence

import pandas as pd
from termcolor import colored
def compute_pmi1(sentence, all_responses, anchor_word_idx):
    # Tokenizing
    words = word_tokenize(sentence.lower())
    # Combine all responses into a single list of tokens
    all_tokens = [word for response in all_responses for word in word_tokenize(str(response).lower())]

    freq_dist = FreqDist(all_tokens)

    # Compute PMI for each word
    pmi_scores = {}
    for i, word in enumerate(words):
        if i != anchor_word_idx:
            # Compute co-occurrence counts
            co_occurrence_count = freq_dist[word]

            # Avoid division by zero by adding a small epsilon value
            epsilon = 1e-10

            # Compute PMI
            pmi = np.log2(co_occurrence_count / ((freq_dist[words[anchor_word_idx]] + epsilon) * (freq_dist[word] + epsilon)))
            
            pmi_scores[word] = pmi

    # Min-Max normalization
    if pmi_scores:
        pmi_min = min(pmi_scores.values())
        pmi_max = max(pmi_scores.values())
        for word in pmi_scores:
            pmi_scores[word] = (pmi_scores[word] - pmi_min) / (pmi_max - pmi_min)

def pmi_scores_to_dataframe(pmi_scores):
    # Convert PMI scores dictionary to DataFrame
    df = pd.DataFrame.from_dict(pmi_scores, orient='index', columns=['PMI'])
    df.index.name = 'Word'
    return df

def display_pmi_scores(pmi_scores):
    # Convert PMI scores to DataFrame
    df = pmi_scores_to_dataframe(pmi_scores)
    
    # Display DataFrame
    print("PMI Scores between Words:")
    print(df)


# Compute PMI
pmi_scores = compute_pmi1(sentence,all_responses, anchor_word_idx)

# Highlight text based on PMI scores
highlighted_sentence = highlight_text(sentence.lower(), pmi_scores)




C:\Users\s184075\AppData\Local\Temp\ipykernel_5600\3802064522.py:91: RuntimeWarning: divide by zero encountered in log2
  pmi = np.log2(co_occurrence_count / ((freq_dist[words[anchor_word_idx]] + epsilon) * (freq_dist[word] + epsilon)))
C:\Users\s184075\AppData\Local\Temp\ipykernel_5600\3802064522.py:100: RuntimeWarning: invalid value encountered in scalar subtract
  pmi_scores[word] = (pmi_scores[word] - pmi_min) / (pmi_max - pmi_min)
C:\Users\s184075\AppData\Local\Temp\ipykernel_5600\3802064522.py:100: RuntimeWarning: invalid value encountered in scalar divide
  pmi_scores[word] = (pmi_scores[word] - pmi_min) / (pmi_max - pmi_min)


TypeError: argument of type 'NoneType' is not iterable

In [15]:
# Print highlighted text
print(highlighted_sentence)
display_pmi_scores(pmi_scores)

tokyo is the capital city of japan 
PMI Scores between Words:
               PMI
Word              
is            -inf
the           -inf
capital  33.219281
city          -inf
of            -inf
japan         -inf


In [ ]:

def compute_pmi(sentence, anchor_word_idx):
    # Tokenize the sentence
    words = word_tokenize(sentence.lower())

    # Remove punctuation tokens
    words = [word for word in words if word.isalnum()]

    # Compute frequency distribution of tokens
    freq_dist = FreqDist(words)

    # Compute PMI for each word
    pmi_scores = {}
    for i, word in enumerate(words):
        if i != anchor_word_idx:
            # Compute co-occurrence counts
            co_occurrence_count = freq_dist[word]

            # Compute PMI
            pmi = np.log2(co_occurrence_count / (freq_dist[words[anchor_word_idx]] * freq_dist[word]))
            pmi_scores[word] = pmi

    return pmi_scores

def highlight_text(sentence, pmi_scores):
    # Tokenize the sentence
    words = word_tokenize(sentence)

    # Highlight words based on PMI scores
    highlighted_sentence = ""
    for word in words:
        if word in pmi_scores:
            # Set highlight color based on PMI score
            pmi_score = pmi_scores[word]
            if pmi_score > 0:
                highlighted_word = f"\033[91m{word}\033[0m"  # Red for positive PMI
            else:
                highlighted_word = f"\033[92m{word}\033[0m"  # Green for negative PMI
        else:
            highlighted_word = word
        highlighted_sentence += highlighted_word + " "

    return highlighted_sentence



# Compute PMI
pmi_scores = compute_pmi(sentence, anchor_word_idx)

# Highlight text based on PMI scores
highlighted_sentence = highlight_text(sentence.lower(), pmi_scores)

# Print highlighted text
print(highlighted_sentence)

In [22]:
def compute_pmi(sentence, all_responses, anchor_word_idx, prompts_per_word):
    # Tokenize the sentence (splits )
    words = word_tokenize(sentence.lower())

    # Clean punctuation and so forth 
    tokens = [token for token in words if token.isalnum()]
    
    # Convert words to lowercase
    words = [word.lower() for word in tokens]

    # Create a DataFrame to store PMI calculations
    pmi_df = pd.DataFrame(columns=words)
    pmi_df.loc['px'] = 0
    pmi_df.loc['py'] = 0
    pmi_df.loc['pxy'] = 0
    pmi_df.loc['pmi'] = 0
    pmi_df.loc['saliency'] = 0

    # variables
    idx_y = 0
    word_x = words[anchor_word_idx].lower()

    epsi = 1e-10 # no division by zero 
    # Iterate over each response
    for i, responses in enumerate(all_responses):
        px = epsi
        py = epsi
        pxy = epsi

        # Check response index matches the anchor word index
        if anchor_word_idx == i:
            idx_y = 1

        word_y = words[i + idx_y].lower()

        # Iterate over each response pair
        for response in responses:
            x = response[0]
            y = response[1]
            if x == word_x:
                px += 1
            if y == word_y:
                py += 1
            if x == word_x and y == word_y:
                pxy += 1

        # Calculate probabilities and PMI
        px = px / prompts_per_word
        py = py / prompts_per_word
        pxy = pxy / prompts_per_word
        pmi = np.log2(pxy / (px * py))

        # Update DataFrame with calculated values
        pmi_df.at['px', word_y] = px
        pmi_df.at['py', word_y] = py
        pmi_df.at['pxy', word_y] = pxy
        pmi_df.at['pmi', word_y] = pmi

    # Set the anchor word column to NaN
    pmi_df[word_x] = np.nan

    # Normalize PMI values and calculate saliency
    min_pmi = np.round(pmi_df.loc['pmi'].min(), 10)
    max_pmi = pmi_df.loc['pmi'].max()
    pmi_df.loc['saliency'] = (pmi_df.loc['pmi'] - min_pmi) / (max_pmi - min_pmi)

    return pmi_df


def highlight_text(sentence, p_df):
    # Tokenize the sentence
    words = word_tokenize(sentence)

    # Highlight words based on saliency scores from p_df
    highlighted_sentence = ""
    for word in words:
        if word in p_df.columns:
            # Get the saliency score for the word
            saliency_score = p_df.loc['saliency', word]

            # Set highlight color based on saliency score
            if saliency_score > 0.1:
                highlighted_word = f"\033[35m{word}\033[0m"  # Bold pink for high saliency
            else:
                highlighted_word = f"\033[91m{word}\033[0m"  # Purple for low saliency
        else:
            highlighted_word = word
        highlighted_sentence += highlighted_word + " "

    return highlighted_sentence

In [23]:
p_df = compute_pmi(sentence, all_responses, anchor_word_idx, prompts_per_word)
print(p_df)

# Highlight text based on saliency scores
highlighted_sentence = highlight_text(sentence.lower(), p_df)

# Print highlighted text
print(highlighted_sentence)

          tokyo        is       the   capital          city            of  \
px          NaN  0.950000  0.900000  0.500000  1.000000e+00  1.000000e+00   
py          NaN  0.950000  0.550000  0.350000  5.500000e-01  1.000000e+00   
pxy         NaN  0.950000  0.550000  0.350000  5.500000e-01  1.000000e+00   
pmi         NaN  0.074001  0.152003  1.000000 -7.213476e-12 -7.213476e-12   
saliency    NaN  0.017122  0.035170  0.231378 -1.669041e-12 -1.669041e-12   

             japan  
px        0.050000  
py        0.050000  
pxy       0.050000  
pmi       4.321928  
saliency  1.000000  
tokyo is the capital city of japan 
